### Tarea 2

In [1]:
import numpy as np
import zipfile
import tarfile
import os
from collections import defaultdict
import nltk
from nltk.util import ngrams
from tqdm import tqdm 
import xml.etree.ElementTree as ET
import re

# Download NLTK data for tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
compressed_file = "Datasets.zip"
with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
    folder_name = os.path.splitext(compressed_file)[0]  # Remove the ".zip" extension
    target_folder = os.path.join(folder_name)
    
    if not os.path.exists(target_folder):
        # Create the folder within the target directory
        os.mkdir(target_folder)

    
        # Extract all files to the target folder
        zip_ref.extractall(target_folder)

print("Extracción completada")

Extracción completada


In [3]:
compressed_file = "Datasets\\20news-18828.tar.gz"
folder_name = os.path.splitext(os.path.splitext(compressed_file)[0])[0]  # Remove the ".tar.gz" extension

if not os.path.exists(folder_name):
    os.mkdir(folder_name)

# Extract all files from the TAR.GZ archive to the target folder without creating an additional subfolder
with tarfile.open(compressed_file, 'r:gz') as tar_ref:
    members = tar_ref.getmembers()
    tar_ref.extractall(path=folder_name, members=members)

print("Extraction completed")

Extraction completed


In [4]:
compressed_file = "Datasets\\BAC\\blogs.zip"
with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
    folder_name = os.path.splitext(compressed_file)[0]  # Remove the ".zip" extension
    target_folder = os.path.join(folder_name)
    
    if not os.path.exists(target_folder):
        # Create the folder within the target directory
        os.mkdir(target_folder)

    
        # Extract all files to the target folder
        zip_ref.extractall(target_folder)

print("Extracción completada")

Extracción completada


In [6]:
# Define the paths to the dataset folders
path_20N = "Datasets\\20news-18828"

# Define the paths to the consolidated files
consolidated_20N_file = "consolidated_20N.txt"

# Function to read and consolidate documents in a folder
def consolidate_documents(folder_path, output_file):
    with open(output_file, "w", encoding="utf-8") as output:
        for root, _, files in os.walk(folder_path):
            for file_name in tqdm(files, desc="Processing Documents"):  # Add tqdm here
                file_path = os.path.join(root, file_name)
                with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                    content = file.read()
                    output.write(content)
                    output.write("\n")  # Add a newline to separate documents

# Consolidate documents from the "20news-18828" dataset with a progress bar
consolidate_documents(path_20N, consolidated_20N_file)
print("Consolidated 20N dataset.")

Processing Documents: 0it [00:00, ?it/s]
Processing Documents: 0it [00:00, ?it/s]
Processing Documents: 100%|██████████| 628/628 [00:00<00:00, 8776.36it/s]

Consolidated 20N dataset.


In [10]:
# Define the path to the BAC dataset folder
path_BAC = "Datasets\\BAC"

# Define the path to the consolidated text file
consolidated_BAC_file = "consolidated_BAC.txt"

# Function to consolidate text from XML documents in a folder
def consolidate_text_from_xml(folder_path, output_file):
    with open(output_file, "w", encoding="utf-8") as output:
        for root, _, files in os.walk(folder_path):
            for file_name in tqdm(files, desc="Processing XML files"):  # Add tqdm here
                file_path = os.path.join(root, file_name)
                # Check if the file is within the "blogs" folder and has a ".xml" extension
                if "blogs" in root and file_name.endswith(".xml"):
                    try:
                        tree = ET.parse(file_path)
                        root_element = tree.getroot()
                        for post_element in root_element.findall(".//post"):
                            content = post_element.text.strip() if post_element.text else ""
                            # Write the extracted content to the output file
                            output.write(content)
                            output.write("\n")  # Add a newline to separate documents
                    except ET.ParseError as e:
                        with open(file_path, "r", encoding="utf-8", errors="ignore") as file_:
                            xml_content = file_.read()

                        # Define a regular expression pattern to match content between <date> and </date> tags
                        pattern = r'<date>.*?</date>'

                        # Use re.sub() to replace matched patterns with an empty string
                        xml_content_filtered = re.sub(pattern, '', xml_content, flags=re.DOTALL)

                        xml_content_filtered = re.sub(r'<.*?>', '', xml_content_filtered)
                        # Remove multiple spaces with a single space
                        xml_content_filtered = re.sub(r'\s+', ' ', xml_content_filtered)

                        xml_content_filtered = xml_content_filtered.replace('\n', '')
                        output.write(xml_content_filtered.strip())
                        output.write("\n")  # Add a newline to separate documents
                        # print(f"Error parsing {file_path}: {e}, using raw file...")

# Path to the "blogs" folder within the BAC dataset
blogs_path_BAC = os.path.join(path_BAC, "blogs")

# Consolidate text content from XML documents within the "blogs" folder of the BAC dataset
consolidate_text_from_xml(blogs_path_BAC, consolidated_BAC_file)
print("Consolidated BAC dataset (text content from XML documents within 'blogs' folder).")

Processing XML files: 0it [00:00, ?it/s]
Processing XML files: 100%|██████████| 19320/19320 [00:47<00:00, 408.31it/s] 

Consolidated BAC dataset (text content from XML documents within 'blogs' folder).
